In [46]:
import pandas as pd

tract_join=pd.read_excel('urws_tracts_intersect.xlsx')
tract_data=tract_join[['PWSID','NAME','D_POPULATI','URWS_Area','GEOID10','Hh_MedianIncome','ALAND10','AWATER10','intArea','urws_percent']]
tract_data.loc[:,'Tract_Frac']=tract_data['intArea']/(tract_data['ALAND10']+tract_data['AWATER10'])
caldata=pd.read_excel('calenviroscreen.xlsx','CES 3.0 (2018 Update)')

C:\Anaconda-3.7\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Anaconda-3.7\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [47]:
calsub=caldata[['Census Tract','Total Population','CES 3.0 Score']]
cal_merge=pd.merge(calsub,tract_data,left_on='Census Tract',right_on='GEOID10')
cal_merge.loc[:,'Tract_Frac_Pop']=cal_merge['Tract_Frac']*cal_merge['Total Population']

In [51]:
#Population weighted values?
sys_stats=pd.DataFrame()
field_calcs=['CES 3.0 Score','Hh_MedianIncome']
for p in cal_merge['PWSID'].unique():
    csub=cal_merge[cal_merge['PWSID']==p]
    cname=csub['NAME'].values[0]
    sys_stats.loc[p,'NAME']=cname
    sys_stats.loc[p,'N']=len(csub)
    sys_stats.loc[p,'Population']=csub['D_POPULATI'].mean()
    if (len(csub)<2):

        sys_stats.loc[p,'Population']=csub['D_POPULATI'].values[0]
        for c in field_calcs:
            sys_stats.loc[p,c]=csub[c].values[0]
    else:

        pop_tracts=csub['Tract_Frac_Pop'].sum()
        for c in field_calcs:
            #print(csub[['Tract_Frac_Pop',c]])
            #print((csub['Tract_Frac_Pop']*csub[c]).sum()/csub['Tract_Frac_Pop'].sum())
            sys_stats.loc[p,c]=(csub['Tract_Frac_Pop']*csub[c]).sum()/csub['Tract_Frac_Pop'].sum()
        

In [50]:
sys_stats.to_excel('example_pop_weighted_characteristics.xlsx')